In [1]:
import os
import sys
sys.path.insert(0, os.getcwd()[:-5])

import pandas as pd
from PIL import Image
import sys
import torch
from torch.utils.data import DataLoader
from torchattacks import FGSM, DeepFool, BIM, RFGSM, PGD, Square, TIFGSM
from torchmetrics import StructuralSimilarityIndexMeasure
from torchvision import transforms

from utils.balancedDataset import BalancedDataset
from utils.const import *
from utils.helperFunctions import *
from utils.nonMathAttacks import NonMathAttacks

In [2]:
ATTACK_NAME = 'FGSM'

TASK = 'catDog'
DATASET = 'bing'
MODEL = 'alexnet'
BALANCE = '20_80'

IMAGE = 'cat_4049.jpg'

epss = [0, 0.1, 0.2, 0.3]

In [3]:
# Ranges and step for attack epsilon

attacksParams = {
    "math": {
        "BIM": {"init": 0.01, "steps": 0.01, "threshold": 0.3},
        "DeepFool": {"init": 10, "steps": 1, "threshold": 100},
        "FGSM": {"init": 0.01, "steps": 0.01, "threshold": 0.3},
        "PGD": {"init": 0.01, "steps": 0.01, "threshold": 0.3},
        "RFGSM": {"init": 0.01, "steps": 0.01, "threshold": 0.3},
        "Square": {"init": 0.1, "steps": 0.05, "threshold": 0.3},
        "TIFGSM": {"init": 0.01, "steps": 0.01, "threshold": 0.3}
    },
    "nonmath": {
        "BoxBlur": {"init": 0.5, "steps": 0.5, "threshold": 10},
        "GaussianNoise": {"init": 0.005, "steps": 0.005, "threshold": 0.1},
        "GreyScale": {"init": 1, "steps": 0, "threshold": 1},
        "InvertColor": {"init": 1, "steps": 0, "threshold": 1},
        "RandomBlackBox": {"init": 10, "steps": 10, "threshold": 200},
        "SaltPepper": {"init": 0.005, "steps": 0.005, "threshold": 0.1},
        "SplitMergeRGB": {"init": 1, "steps": 0, "threshold": 1}
    }
}

nonMathAttacks = NonMathAttacks()


## Generating same image at different eps

In [4]:
currentAttackParams = attacksParams["math"][ATTACK_NAME]
# datasetDir = os.path.join(DATASETS_DIR, DATASET)
datasetDir = '../datasets/' + TASK + '/' + DATASET
testDir = os.path.join(datasetDir, "test")

toTensor = transforms.Compose([transforms.ToTensor()])
toNormalizedTensor = transforms.Compose([
    transforms.Resize(INPUT_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(
        NORMALIZATION_PARAMS[0], NORMALIZATION_PARAMS[1])
])


In [5]:
path = '../models/' + TASK + '/' + DATASET + '/' + MODEL + '/' + MODEL + '_' + BALANCE + '.pt'

modelData = torch.load(path, map_location=torch.device('cpu'))

modelDataset = modelData["dataset"]
modelName = modelData["model_name"]

torch.cuda.empty_cache()

modelPercents = "_".join([str(x)
                          for x in modelData["balance"]])
model = modelData["model"].to(DEVICE)

# Test dataset without normalization (for generating samples)
originalTestDataset = BalancedDataset(
    testDir, transform=toTensor, datasetSize=DATASET_SIZE, use_cache=False, check_images=False, with_path=True)

setSeed()
originalTestDataLoader = DataLoader(
    originalTestDataset, batch_size=16, num_workers=0, shuffle=False)

# Test dataset with normalization (for evaluation)
testDataset = BalancedDataset(
    testDir, transform=toNormalizedTensor, datasetSize=DATASET_SIZE, use_cache=False, check_images=False, with_path=True)

setSeed()
testDataLoader = DataLoader(
    testDataset, batch_size=16, num_workers=0, shuffle=False)


In [6]:
for eps in epss:
    attacks = {
        "BIM": BIM(model, eps=eps),
        "DeepFool": DeepFool(model, overshoot=eps),
        "FGSM": FGSM(model, eps=eps),
        "PGD": PGD(model, eps=eps),
        "RFGSM": RFGSM(model, eps=eps),
        "Square": Square(model, eps=eps),
        "TIFGSM": TIFGSM(model, eps=eps)
    }
    for attack in attacks:
        if attack == ATTACK_NAME:
            attacker = attacks[attack]

            for images, labels, paths in originalTestDataLoader:
                adversarials = attacker(images, labels)

                for adversarial, label, fName in zip(adversarials, labels, paths):
                    if os.path.basename(fName) == IMAGE:
                        image = transforms.ToPILImage()(adversarial).convert("RGB")

                        imageName = os.path.basename(fName).split('.')[0] + '_' + str(eps) + '.jpg'
                        savePath = './figures/images/'

                        if not os.path.exists(savePath):
                            os.makedirs(savePath)

                        image.save(os.path.join(savePath, imageName), "JPEG")
